# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [55]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [56]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

614

In [57]:
#print cities for sanity check
print(cities)

['kavaratti', 'nome', 'albany', 'punta arenas', 'tuktoyaktuk', 'shkurinskaya', 'provideniya', 'havoysund', 'ponta do sol', 'banda aceh', 'mumford', 'flinders', 'kamenskoye', 'praia da vitoria', 'thompson', 'longyearbyen', 'rikitea', 'samarai', 'ialibu', 'vaini', 'ushuaia', 'hobart', 'foam lake', 'sao jose da coroa grande', 'narsaq', 'yellowknife', 'airai', 'port elizabeth', 'shache', 'bluff', 'clyde river', 'byron bay', 'new norfolk', 'san francisco', 'carauari', 'takoradi', 'osypenko', 'zhigansk', 'grand centre', 'zhuanghe', 'rungata', 'busselton', 'tasiilaq', 'qaanaaq', 'tupik', 'jamestown', 'iqaluit', 'illoqqortoormiut', 'hervey bay', 'bengkulu', 'walvis bay', 'saint anthony', 'buba', 'havre-saint-pierre', 'avarua', 'dingle', 'vanavara', 'nhulunbuy', 'aripuana', 'port macquarie', 'bathsheba', 'sao domingos', 'saint-francois', 'lagoa', 'nanortalik', 'butaritari', 'kalmunai', 'necochea', 'jiangkou', 'vestmannaeyjar', 'victoria', 'fort nelson', 'maragogi', 'gaya', 'leshukonskoye', 'bor

In [58]:
#weather checks
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

query_url = f"{url}appid={weather_api_key}&units={units}&q="

#loop set up
lat=[]
lng=[]
max_temp=[]
humidity=[]
cloudiness=[]
windspeed=[]
country=[]
date=[]
city_sample = ['hilo', 'samusu', 'caravelas', 'castro', 'bredasdorp', 'albany']

#loop
for city in city_sample:
    try:
        response = requests.get(query_url + city).json()
        print(f'processing record for: {city}')
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lng'])
        max_temp.append(response['main']['temp_max'])
        cloudiness.append(response['weather']['description'])
        windspeed.append(response['wind']['speed'])
        country.append(response['sys']['country'])
        date.append(response['dt'])

    except KeyError:
        print("city record not found...skipping..")

print('done')


SyntaxError: invalid character in identifier (<ipython-input-58-d15975371e3f>, line 19)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
#df creation!!!!!!!!!!!!!!!
# weather_dict = {
    #"city": cities,
    #"lat": lat,
    #"temp": temp
#}
#weather_data = pd.DataFrame(weather_dict)
#weather_data.head()

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression